In [1]:
import re
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import emoji

In [7]:
import chardet
with open('data/classified_emojis.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'Windows-1254',
 'confidence': 0.3226829664768674,
 'language': 'Turkish'}

In [2]:
twitter_file = open("data/raw_korean.txt", "r", encoding = "utf-8")
twitter = twitter_file.read()
twitter_file.close()

twitter[:500]

'RT @haha247979: @onemoreonce4bts @BTS_twt 넵넵💜 저..  늦게 해시 투표 동참한거 사실.. 조금.. 많이 죄송하긴 해써요😂😂\n열심히 슨배님덜 따라갈게요~😍\n\nButter by BTS #BTS\nStream #BTS_B…@green_zzang 맞아맞아 그렇긴 하지! 😂 근데 난 내용이 맘에 안 들거나 더 안 읽어도 되겠다 싶으면 다 안 읽었는데도 읽은 책처럼 생각해(!) ㅋㅋㅋㅋㅋ@weareDRIPPIN 오늘의 한마디를 주세요!😂\n\n#DRIPPIN #드리핀 #Villain #드리핀과_함께하는_멘션파티@KnitOhlala 이거 때문에 어제 실 주문을 한지라 ㅋㅋㅋㅋ 매우 찔립니다요!!! 근데 넘 잘 입을 것 같아서 함뜨참가! 얍! 색조합 잘 못하는데 원작하고 비슷하게 갑니다 😂@shotasaigo 까까님😂😂 생일메세지 정말 감사해요 🥺🥺💙 끝까지 덕질해라는 뜻인가봐요😇 까까님도 남은 시간 즐겁게 보내세요 🥰🥰아 내스급 성현제 책갈피가 아니라 옷핀입니다😂😂'

In [16]:
classified_emojis = pd.read_csv('data/classified_emojis.csv', encoding = 'utf16')
# classified_emojis = open('data/classified_emojis.csv', encoding = 'utf-8', errors="ignore")
# with open(path, 'rb') as f:
#   text = f.read()

classified_emojis.drop_duplicates(subset = ['emojis'], ignore_index = True)
classified_emojis

UnicodeError: UTF-16 stream does not start with BOM

In [ ]:
#big data crawled with academic api added
twitter_file_big = open("data/raw_korean_2015_2021.txt", "r", encoding = "utf-8")
twitter += twitter_file_big.read()
twitter_file_big.close()

twitter[:500]

In [ ]:
# kaggle_file = open("data/kaggle_translated.txt", "r", encoding = "utf-8")
# kaggle = kaggle_file.read()
# kaggle = kaggle.replace("⁇", "")
# kaggle_file.close()

# kaggle[:500]

In [ ]:
# text = twitter + kaggle
text = twitter

In [ ]:
#영문, 특수문자(? ! . 제외), 숫자 제거
text = re.sub('[a-zA-z]','',text)
text = re.sub('[\{\}\[\]\/;:|\)*~`^\-_+<>@\#$%&\\\=\(\'\"\♥\♡\n0-9]','',text)
text = text.strip()
text[-500:]

In [ ]:
emojis = emoji.emoji_lis(text)
emojis[:10]

In [ ]:
emojis = list(set([i['emoji'] for i in emojis]))
print(len(emojis))
emojis[:10]

In [ ]:
delimiters = ['.', '!', '?', 'ㅎ', 'ㅋ', ' ']
delimiters.extend(emojis)

#두번이상 반복된 것 한 번으로 축소
for d in delimiters:
    flag = False
    while True:
        if text.find(d+d) == -1:
            break
        text = text.replace(d+d, d)

text[:500]

In [ ]:
df = df(columns = ['x', 'y'])
df

In [ ]:
classified_emojis[classified_emojis['emojis'] == '😍']['class'].head(1).item()

In [ ]:
len(text)

In [ ]:
delimiters.remove(' ')
x = ""

for i in range(len(text)):
    if text[i] in delimiters:
#         if i+1 < len(text) and text[i+1] in delimiters:
#             continue
        if len(x) < 10:
            continue
        if text[i] not in emojis:
          #  df = df.append({'x': x, 'y': 'N'}, ignore_index = True)
            x = ""
            continue
        elif text[i] in list(classified_emojis['emojis']):  # classified_emojis에 없는 것은 무시한다.
            emoji = classified_emojis[classified_emojis['emojis'] == text[i]]['class'].head(1).item()
            df = df.append({'x': x, 'y': emoji}, ignore_index = True)
        x = ""
    else:
        x += text[i]

In [ ]:
df.head()

In [ ]:
len(df) # -> 30988 166062

In [ ]:
#중복 제거
df = df.drop_duplicates(subset = ['x']).reset_index(drop = True)

In [ ]:
print(len(df))
df.head() #2231 -> 5725 -> 64573

In [ ]:
#x; 한국어만 남기기
df['x'] = [re.compile('[^ ㄱ-ㅣ가-힣]+').sub('', df['x'][i]).strip() for i in range(len(df['x']))]

In [ ]:
len(df) #total data length -> 15715

In [ ]:
df.to_csv('data/twitter_clean_all.csv', index=False)

In [ ]:
df_count = df['y'].value_counts(sort = True)
len(df_count) #unique labels
df_count

In [ ]:
#count 10 이하인 것 제거
# more_than_ten = df_count[df_count > 10]

#라벨 수를 최빈도 순 줄인다.
frequent_labels = list(df_count.head(5).index)
df_sort = df[df['y'].isin(frequent_labels)]
df_sort.head(10)

In [ ]:
len(df_sort) #total data length 1261 -> 6166 -> 40233

In [ ]:
df_count = df_sort['y'].value_counts(sort = True)
print(len(df_count))
df_count

# {love}                141
# {kind-smile}          119
# {laughing-out}         96
# {open-mouth-smile}     84
# {good-job}             81
# Name: y, dtype: int64

# {love}                1537
# {kind-smile}          1355
# {solid-sad}           1173
# {open-mouth-smile}    1148
# {good-job}             953
# Name: y, dtype: int64

In [ ]:
# #undersample data with label 'N'
# tail_len = int(len(df[df['y'] == 'N']) * 0.9)
# to_drop = df[df['y']=='N'].tail(tail_len)
# df = df.drop(to_drop.index) 
# df.reset_index(drop = True)

In [ ]:
# df_count = df['y'].value_counts(sort = True)
# print(len(df_count))
# df_count

In [ ]:
# print(len(df))
# df.tail(10)

In [ ]:
df_sort.to_csv('data/twitter_clean.csv', index=False)